<a href="https://colab.research.google.com/github/danatol/Data-Analysis/blob/master/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22%D0%92%D0%B5%D0%B1%D0%B8%D0%BD%D0%B0%D1%80_25_%D0%B0%D0%BF%D1%80%D0%B5%D0%BB%D1%8F_%D0%94%D0%BE%D0%BE%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D0%B5_chatGPT%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Установка библиотек. Сервисные функции
!pip -q install --upgrade tiktoken
!pip -q install langchain openai chromadb
!pip -q install gspread oauth2client

from langchain.llms import OpenAI
from langchain.docstore.document import Document
import requests
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate
import pathlib
import subprocess
import tempfile
import ipywidgets as widgets
import os
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import re

import os
import openai
import tiktoken
import re


class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

class GPT():
  def __init__(self):
    pass

  @classmethod
  def set_key(cls):
      password_input = widgets.Password(
          description='Введите пароль:',
          layout=widgets.Layout(width='500px'),
          style={'description_width': 'initial', 'white-space': 'pre-wrap', 'overflow': 'auto'})
      login_button = widgets.Button(description='Авторизация')
      output = widgets.Output()

      def on_button_clicked(_):
          with output:
              openai.api_key = password_input.value
              os.environ["OPENAI_API_KEY"] = openai.api_key
              print(f'{bcolors.OKGREEN}{bcolors.BOLD}Ключ сохранен!{bcolors.ENDC}')
              password_input.layout.display = 'none'
              login_button.layout.display = 'none'

      login_button.on_click(on_button_clicked)
      display(widgets.VBox([password_input, login_button, output]))

  def load_search_indexes(self, url: str) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text
    return self.create_embedding(text)

  def load_prompt(self,
                  url: str) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text
    return f'{text}'


  def create_embedding(self, data):
    def num_tokens_from_string(string: str, encoding_name: str) -> int:
      """Returns the number of tokens in a text string."""
      encoding = tiktoken.get_encoding(encoding_name)
      num_tokens = len(encoding.encode(string))
      return num_tokens

    source_chunks = []
    splitter = CharacterTextSplitter(separator="\n", chunk_size=1024, chunk_overlap=0)

    for chunk in splitter.split_text(data):
      source_chunks.append(Document(page_content=chunk, metadata={}))

    # Создание индексов документа
    search_index = Chroma.from_documents(source_chunks, OpenAIEmbeddings(), )

    count_token = num_tokens_from_string(' '.join([x.page_content for x in source_chunks]), "cl100k_base")
    print('\n ===========================================: ')
    print('Количество токенов в документе :', count_token)
    print('ЦЕНА запроса:', 0.0004*(count_token/1000), ' $')
    return search_index

  def answer(self, system, topic, temp = 1):
    messages = [
      {"role": "system", "content": system},
      {"role": "user", "content": topic}
      ]

    completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=messages,
      temperature=temp
      )

    return completion.choices[0].message.content

  def num_tokens_from_messages(self, messages, model="gpt-3.5-turbo-0301"):
    """Returns the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        encoding = tiktoken.get_encoding("cl100k_base")
    if model == "gpt-3.5-turbo-0301":  # note: future models may deviate from this
        num_tokens = 0
        for message in messages:
            num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
            for key, value in message.items():
                num_tokens += len(encoding.encode(value))
                if key == "name":  # if there's a name, the role is omitted
                    num_tokens += -1  # role is always required and always 1 token
        num_tokens += 2  # every reply is primed with <im_start>assistant
        return num_tokens
    else:
        raise NotImplementedError(f"""num_tokens_from_messages() is not presently implemented for model {model}.
See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens.""")

  def insert_newlines(self, text: str, max_len: int = 170) -> str:
    words = text.split()
    lines = []
    current_line = ""
    for word in words:
        if len(current_line + " " + word) > max_len:
            lines.append(current_line)
            current_line = ""
        current_line += " " + word
    lines.append(current_line)
    return "\n".join(lines)


  def dialog(self):
    user = ''
    dialog = ''

    print(f'{bcolors.OKBLUE}{bcolors.BOLD}С чем связан ваш интерес к искусственному интеллекту?{bcolors.ENDC}')

    while user.lower() not in ['stop', 'exit', 'выход']:
      user = input('Клиент: ')
      if (user == 'Stop'): break

      dialog += '\n\n' + 'Клиент: ' + user
      add_dialog = gpt.answer(expert_prompt, user)

      dialog += '\n\n' + 'Менеджер: ' + add_dialog
      print(f'\n{bcolors.OKBLUE}{bcolors.BOLD}Менеджер:{bcolors.ENDC} {self.insert_newlines(add_dialog)}')
      report = gpt.answer(validation_prompt, dialog)
      answer = gpt.answer(action_prompt, report)

      print(f'\n{bcolors.OKGREEN}{bcolors.BOLD}Отчёт системы:\n {bcolors.ENDC}{report}')
      print(f'\n{bcolors.HEADER}{bcolors.BOLD}Менеджер: {bcolors.ENDC}{self.insert_newlines(answer)}\n\n')

    return dialog

  def answer_index(self, system, topic, search_index, temp = 1, verbose = 0):

    #Выборка документов по схожести с вопросом
    docs = search_index.similarity_search(topic, k=5)
    if (verbose): print('\n ===========================================: ')
    message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\nОтрывок документа №{i+1}\n=====================' + doc.page_content + '\n' for i, doc in enumerate(docs)]))
    if (verbose): print('message_content :\n ======================================== \n', message_content)

    messages = [
      {"role": "system", "content": system + f"{message_content}"},
      {"role": "user", "content": topic}
      ]

    # example token count from the function defined above
    if (verbose): print('\n ===========================================: ')
    if (verbose): print(f"{self.num_tokens_from_messages(messages, 'gpt-3.5-turbo-0301')} токенов использовано на вопрос")

    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=messages,
    temperature=temp
    )
    if (verbose): print('\n ===========================================: ')
    if (verbose): print(f'{completion["usage"]["total_tokens"]} токенов использовано всего (вопрос-ответ).')
    if (verbose): print('\n ===========================================: ')
    if (verbose): print('ЦЕНА запроса с ответом :', 0.002*(completion["usage"]["total_tokens"]/1000), ' $')
    if (verbose): print('\n ===========================================: ')
    print('ОТВЕТ : \n', self.insert_newlines(completion.choices[0].message.content))

    # return completion

  def get_chatgpt_ansver3(self, system, topic, search_index, temp = 1):

    messages = [
      {"role": "system", "content": system},
      {"role": "user", "content": topic}
      ]

    completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=messages,
      temperature=temp
      )
    print('ОТВЕТ : \n', self.insert_newlines(completion.choices[0].message.content))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 6.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 938.7/938.7 kB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 118.8 MB/

In [ ]:
gpt = GPT()

In [ ]:
GPT.set_key()

<h3><b> Маркетинг: ответы на вопросы</b></h3>

In [ ]:
#Все маркетинговые материалы
marketing_index = gpt.load_search_indexes('https://docs.google.com/document/d/1aM6Dd-RwxG2J9LovrX6lJFuk8vmruu54zvauTgFWxCk/edit')

In [ ]:
marketing_chat_promt = '''Ты менеджер поддержки в чате компании, компания продает курсы по AI.
У тебя есть большой документ со всеми материалами о продуктах компании.
Тебе задает вопрос клиент в чате, дай ему ответ, опираясь на документ, постарайся ответить так, чтобы человек захотел после ответа купить обучение.
и отвечай максимально точно по документу, не придумывай ничего от себя.
Документ с информацией для ответа клиенту: '''

In [ ]:
gpt.answer_index(
    marketing_chat_promt,
    'Расскажи про тариф Продвинутый',
    marketing_index
)

In [ ]:
gpt.answer_index(
    marketing_chat_promt,
    'Расскажи про тариф Продвинутый',
    marketing_index,
    verbose = 1
)


 ===========================================: 
message_content :
 
Отрывок документа №1
=====================Тариф содержит широкую базу, включая основы нейронных сетей, обнаружение объектов, трансформеры, генетические алгоритмы, распознавание речи, обучение с подкреплением,интеграцию в продакшн, чат-боты, генеративные модели и сегментацию. Однако в него включено только 32 базовых урока. Тариф подходит для студентов, которые лимитированы в деньгах. Сохраняя низкую стоимость тарифа, УИИ может привлечь большее количество студентов и создать большой пул потенциальных проектов. Конечная цель - предоставить студентам множество возможностей для участия в значимых проектах, которые будут способствовать их профессиональному росту. На тариф предусмотрена рассрочка на 2-3 года по 2500 рублей в месяц.


Тариф “Продвинутый” - самый крутой тариф.


      * обучение 3 года и 8 месяцев;
      * доступ ко всему контенту;
      * 11 курсов;
      * 2 года поддержки куратора;
      * гарантия трудоустр

In [ ]:
marketing_chat_promt_logic = '''Ты менеджер поддержки в чате компании, компания продает курсы по AI.
Ты максимально логичный человек, который отвечает структурированно, коротко и фактами.
У тебя есть большой документ со всеми материалами о продуктах компании.
Тебе задает вопрос клиент в чате, дай ему ответ, опираясь на документ, постарайся ответить так, чтобы человек захотел после ответа купить обучение
 и отвечай максимально точно по документу, не придумывай ничего от себя.
 Документ с информацией для ответа клиенту: '''

In [ ]:
gpt.answer_index(
    marketing_chat_promt_logic,
    'Расскажи про тариф Продвинутый',
    marketing_index
)

ОТВЕТ : 
  Тариф "Продвинутый" включает в себя 11 курсов и предоставляет поддержку куратора в течение 2 лет. Также мы гарантируем трудоустройство и помогаем в реализации AI
 проектов. Кроме того, данный тариф предоставляет 4 стажировки, что дает возможность приобрести ещё больше навыков. Стоимость тарифа без учета скидки составляет 219 900
 рублей, но с использованием скидки 30% он доступен всего за 149 900 рублей. К тому же, мы предоставляем возможность рассрочки на 12, 24 или 36 месяцев, что позволяет
 выбрать удобный для вас вариант оплаты. Если у вас возникнут дополнительные вопросы по данному или другим тарифам, я буду рад помочь вам в чате!


In [ ]:
marketing_chat_promt_emotional = '''Ты менеджер поддержки в чате компании, компания продает курсы по AI.
Ты максимально эмоциональный человек, который старается всегда вдохновить собеседника темой AI и вдохновить на изменения.
У тебя есть большой документ со всеми материалами о продуктах компании.
Тебе задает вопрос клиент в чате, дай ему ответ, опираясь на документ, постарайся ответить так, чтобы человек захотел после ответа купить обучение и отвечай максимально точно по документу, не придумывай ничего от себя.
Документ с информацией для ответа клиенту: '''

In [ ]:
gpt.answer_index(
    marketing_chat_promt_emotional,
    'Расскажи про тариф Продвинутый',
    marketing_index
)

ОТВЕТ : 
  Курс Продвинутый - это один из наших наиболее полных и серьезных курсов по искусственному интеллекту. Он включает в себя 11 курсов и обеспечивает 2 года поддержки
 куратором. Также мы гарантируем трудоустройство после прохождения курса и реализацию AI проекта. Стоимость курса составляет 149 900 рублей, однако по акции вы можете
 получить скидку в 30% от этой суммы. Таким образом, вы сможете приобрести курс Продвинутый всего за 104 930 рублей. Во время курса мы предоставим вам 4 стажировки, а
 также консультации по дипломному проекту. При желании вы можете воспользоваться рассрочкой на 12, 24 или 36 месяцев.


In [ ]:
gpt.answer_index(
    marketing_chat_promt,
    'Расскажи про стажировки',
    marketing_index
)

ОТВЕТ : 
  Наша компания предлагает стажировки для тех, кто хочет прокачать свои знания по технологиям искусственного интеллекта, и при этом получить реальный опыт работы в крупных
 компаниях во время обучения. За время стажировки у вас будет конкретная задача, которую нужно решить для конкретного клиента, что позволит получить ценный опыт работы в
 команде, а также заряжаться от прикладной работы в сфере AI. Проекты-задачи, которые выполняют наши студенты на стажировках, реальны и обеспечивают практикой в том, что
 требуется в рыночных проектах. Это позволит вам собрать хороший портфолио и выделиться среди других соискателей на рынке труда. Кроме того, мы помогаем нашим студентам с
 подбором вакансий и прохождением собеседований, что обеспечивает быстрое трудоустройство.


In [ ]:
gpt.answer_index(
    marketing_chat_promt,
    'Расскажи про выкуп стоимости обучения',
    marketing_index
)

ОТВЕТ : 
  Выкуп стоимости обучения - это уникальная возможность, которую мы предлагаем нашим клиентам. Вы можете приобрести один из наших курсов, оплатив его в рассрочку на 3
 года. При этом вы получаете возможность проходить обучение у нас в течение 7 месяцев с гарантированной возможностью получать доход, который уже через 8 месяцев окупит
 ваше обучение. Оплачивая курс в рассрочку, вы платите всего 4.442 рублей в месяц, и всего за 7 платежей вы заплатите 31.094 рублей. Хочу отметить, что у нас есть
 ограниченное количество мест на выкуп стоимости обучения, поэтому стоит поторопиться и оставить предзаказ, чтобы не упустить свою возможность.


In [ ]:
#Новая база знаний от копирайтеров
marketing_new_index = gpt.load_search_indexes('https://docs.google.com/document/d/1sXHb1dYVwyH4M2kFrCk_7TQYZqhDgZXY-xjCIpolA4s/edit')


 ===========================================: 
Количество токенов в документе : 43129
ЦЕНА запроса: 0.0172516  $


In [ ]:
gpt.answer_index(
    marketing_chat_promt,
    'Расскажи про стажировки',
    marketing_new_index
)

ОТВЕТ : 
  Конечно! Для нас стажировки очень важны, так как они дают студентам возможность получить реальную практику, работая в команде и над реальными проектами в ведущих
 IT-компаниях. В программу стажировок входит работа над реальным проектом, индивидуальные учебные треки и еженедельные zoom-звонки с руководителем группы. Всего у нас 4
 стажировки в крупных компаниях, таких как Камаз, Ростелеком, РУДН, Самолет и многие другие. После успешного окончания стажировки студенты получают оффер и выходят на
 работу во время стажировки. Также в УИИ есть также возможность студентам выступить в роли компании, написав техническую заявку и передавая вакансии. Всё это позволяет
 нашим студентам получить те знания и навыки, которые необходимы им для успешной карьеры в IT-сфере и обрести большое количество эмоций и драйва.


In [ ]:
gpt.answer_index(
    marketing_chat_promt,
    'Расскажи про выкуп стоимости обучения',
    marketing_new_index
)

ОТВЕТ : 
  Выкуп стоимости обучения - это возможность устроиться на работу в Университет Искусственного Интеллекта после окончания обучения и возвращения стоимости курса с
 процентами за весь период обучения. Эта гарантия доступна только для тех студентов, которые закончили все курсы и успешно прошли их. Таким образом, студент получает не
 только опыт на самом высоком уровне, но и возможность вернуть вложенные средства. Это означает, что Университет Искусственного Интеллекта заинтересован в успехе своих
 выпускников и предоставляет им реальную перспективу трудоустройства и возврата средств за обучение.


<h3><b> Генерация постов для тг</b></h3>

In [ ]:
#Только посты в телеграм
marketing_tg_post_index = gpt.load_search_indexes('https://docs.google.com/document/d/1SukTbi018v4H8D8Lt9YM9i3Zz_BjEWxNw7aw6T3euo4/edit?usp=sharing')


 ===========================================: 
Количество токенов в документе : 56119
ЦЕНА запроса: 0.0224476  $


In [ ]:
marketing_tg_post_promt = '''Ты эксперт и спикер по AI, ты пишешь посты в телеграм канал, твоя задача делать вдохновляющие, но в то же время конкретные и полезные посты.
Компания, в которой ты работаешь, продает курсы по AI.
У тебя есть большой документ с десятками твоих предыдущих постов и информацией о продуктах компании.
Тебе задает тему твой руководитель, ты должен написать пост в телеграм канал на эту тему.
Бери информацию для поста точно по документу, не придумывай ничего от себя.
Ты должен сохранить стиль написания такой же, как в постах в документе.
Документ с информацией для ответа: '''

In [ ]:
gpt.answer_index(
    marketing_tg_post_promt,
    'Напиши пост про AutoML, почему надо отказываться от обычного написания нейронок и переходить на AutoML',
    marketing_tg_post_index
)

ОТВЕТ : 
  Привет, друзья! Сегодня я хочу рассказать вам о технологии, которая с каждым днем становится все более популярной в области искусственного интеллекта. Это AutoML -
 автоматическое создание нейронных сетей, которое с каждым днем приобретает все большую популярность в различных сферах деятельности. Одной из главных проблем при
 создании нейронных сетей является необходимость тратить много времени и сил на построение правильной архитектуры. Это может занять от нескольких недель до нескольких
 месяцев, в зависимости от сложности задачи и опыта разработчика. Автоматизация этого процесса может значительно упростить жизнь разработчиков и ускорить процесс создания
 нейронных сетей. AutoML использует алгоритмы машинного обучения и генетические алгоритмы для автоматической генерации архитектуры нейронной сети, построения оптимальной
 модели и ее оптимизации. Это означает, что вам больше не нужно писать нейронные сети "вручную", и вы можете сосредоточиться на более важных аспектах свое

In [ ]:
gpt.answer_index(
    marketing_tg_post_promt,
    'Напиши пост про курс по интеграции в production, сделай пост продающим и подробно раскрой выгоды прохождения курса по пунктам',
    marketing_tg_post_index
)

ОТВЕТ : 
  Привет, друзья! Сегодня я хочу рассказать вам о нашем курсе по интеграции в production, который поможет вам стать настоящим профессионалом в области искусственного
 интеллекта. Почему этот курс так важен? Все просто – при работе с нейронными сетями, объем компетенции играет очень большую роль. Чем больше вы умеете, тем проще вы
 сможете выполнить проект, продать его или найти работу. Давайте рассмотрим все плюсы, которые вы получите после прохождения нашего курса: 1. Понимание принципов работы с
 инструментами контроля качества. Вы научитесь создавать различные пайплайны и управлять ими, а также научитесь тестировать приложения. 2. Большой плюс – вы научитесь
 создавать Docker-контейнеры и управлять ими. Это очень важный навык в наше время, который сильно упрощает процесс развертывания проектов в production. 3. Обучение
 методике непрерывной доставки. Вы научитесь создавать пулл-реквесты и настраивать процесс непрерывной интеграции. 4. Интеграция с сервисами облака. Вы узнае

<h3><b> Генерация заголовков таргетированной рекламы</b></h3>

In [ ]:
#Только страницы сайта
marketing_targeting_index = gpt.load_search_indexes('https://docs.google.com/document/d/1puW6Ckm2e5Kc59UKOsnVsHcNXgYlgftFaM4SjI57WdM/edit?usp=sharing')


 ===========================================: 
Количество токенов в документе : 4440
ЦЕНА запроса: 0.0017760000000000002  $


In [ ]:
marketing_targeting_promt = '''Ты специалист настройки таргетированно рекламы, компания, в которой ты работаешь, продает курсы по AI.
У тебя есть большой документ с информацией о продуктах компании.
Твоя задача писать заголовки таргетированной рекламы с акцентами, который задаёт пользователь в запросе.
Бери информацию для заголовка рекламы точно по документу, не придумывай ничего от себя.
Документ с информацией для генерации заголовка: '''

In [ ]:
gpt.answer_index(
    marketing_targeting_promt,
    'Напиши 3 заголовка для таргетированной рекламы',
    marketing_targeting_index
)

ОТВЕТ : 
  1. "Станьте AI разработчиком и зарабатывайте 150.000р в месяц - новый курс от компании по AI обучению" 2. "Учитесь без опыта в программировании: создавайте нейронки с
 помощью курсов компании на русском языке" 3. "Откройте для себя самую большую базу AI контента в мире и станьте экспертом в компьютерном зрении, обработке естественного
 языка и не только"


In [ ]:
gpt.answer_index(
    marketing_targeting_promt,
    'Напиши 3 заголовка для таргетированной рекламы с упором на трудоустройство',
    marketing_targeting_index
)

ОТВЕТ : 
  1. "Станьте AI разработчиком и получайте отличную зарплату уже сейчас!" 2. "Ваш шанс начать карьеру в AI: обучение с практической направленностью" 3. "Приглашаем
 присоединиться к команде нейросетевых профессионалов – обеспечим трудоустройство"


In [ ]:
gpt.answer_index(
    marketing_targeting_promt,
    'Напиши 3 заголовка для таргетированной рекламы с упором на создание AI проекта',
    marketing_targeting_index
)

ОТВЕТ : 
  1. "Создай свой первый AI проект под руководством опытных кураторов" 2. "Станьте AI разработчиком и реализуйте свои идеи в дипломном проекте" 3. "Уникальный курс с
 созданием собственного AI проекта под руководством практиков"


In [ ]:
gpt.answer_index(
    marketing_targeting_promt,
    'Напиши 3 заголовка для таргетированной рекламы с упором на то, что AI - это очень интересно',
    marketing_targeting_index
)

ОТВЕТ : 
  1. "Увлекательное изучение AI с укусом реальности - только у нас!" 2. "Открой для себя захватывающий мир AI и нейронных сетей с нами!" 3. "Погрузись в увлекательный мир
 искусственного интеллекта с нашими курсами AI!"


<h3><b>Менеджер поддержки HR отдела</b></h3>

In [ ]:
#HR база
hr_index = gpt.load_search_indexes('https://docs.google.com/document/d/1ZjpABLZD9BFDjPvYFcwOVcH5nnwx1lkpUCLneBUrzIQ/edit')


 ===========================================: 
Количество токенов в документе : 7972
ЦЕНА запроса: 0.0031888000000000003  $


In [ ]:
hr_promt = '''Ты менеджер поддержки hr отдела, к тебе могут обращаться hr менеджеры и рекрутеры за подсказками и ответами на их вопросы в чате компании.
Постарайся дать развернутый ответ, твоя задача ответить так, чтобы у менеджера не осталось больше вопросов к тебе.
Отвечай по существу, без лишних эмоций и слов, от тебя нужна только точная информация.
Отвечай максимально точно по документу, не придумывай ничего от себя.
Документ с информацией для ответа клиенту: '''

In [ ]:
gpt.answer_index(
    hr_promt,
    'Как выбрать заголовок вакансии',
    hr_index
)

ОТВЕТ : 
  Для того, чтобы привлечь подходящих кандидатов на вакансию, необходимо выбрать подходящие заголовки. Вот инструкция: 1. Придумайте 5-10 вариантов заголовков вакансий,
 содержащих различные ключевые слова, по которым кандидат мог бы найти вакансию. 2. Зайдите в "Поиск" на hh.ru 3. Введите первое название вакансии. 4. В меню справа,
 настройте регион и специализацию в соответствии с требованиями вакансии. 5. Под строкой поиска будет указано количество резюме и соискателей, соответствующих заданным
 параметрам. 6. Откройте 5-10 резюме соискателей и изучите их опыт работы и обязанности, проверьте, насколько соискатели релевантны вашей вакансии. 7. Пройти шаги 1-5
 заново с каждым заголовком, до тех пор, пока не будет найдено название, удовлетворяющее обоим критериям. 8. Если вы уже опубликовали вакансию, но хотите сменить название
 на более подходящее, то примеры названий можно также посмотреть на странице вашей вакансии - в подходящих резюме. Для этого нужно зайти на страницу в

In [ ]:
gpt.answer_index(
    hr_promt,
    'Как оплатить базу кандидатов',
    hr_index
)

ОТВЕТ : 
  Для оплаты базы кандидатов на hh.ru, необходимо выполнить следующие шаги: 1. Перейти на страницу hh.ru, зайти в свой аккаунт и выбрать в меню "Прайс-лист" пункт "База
 резюме". 2. Выбрать нужный тариф и нажать на кнопку "Купить". 3. Заполнить все необходимые поля для оплаты на странице оплаты. Обязательно проверить правильность
 введенной информации. 4. Выбрать способ оплаты, в зависимости от региона и наличия доступных способов оплаты. Варианты могут включать оплату банковской картой,
 электронными деньгами или банковским переводом. 5. Подтвердить платеж и дождаться подтверждения оплаты. 6. Сохранить полученный чек на оплату и направить его человеку,
 отвечающему за бюджеты/оплату. 7. После успешной оплаты базы кандидатов, можно скачивать резюме кандидатов в разных форматах.


In [ ]:
gpt.answer_index(
    hr_promt,
    'Как настроить таргет hh',
    hr_index
)

ОТВЕТ : 
  Чтобы настроить таргетинг на HH.ru, нужно выполнить следующие действия: 1. Зайти на https://clickme.hh.ru/new/dashboard/ 2. Создать новую кампанию, если у Вас еще нет
 активных кампаний. 3. Выбрать в разворачивающемся списке в строке "Вакансия" ту вакансию, на которую кандидаты будут переходить, кликнув на объявление. Строка "Ссылка"
 заполнится автоматически. 4. В строке "Заголовок" пишем название вакансии, которое будет отображено в объявлении. Оно может быть коротким, но максимально отражающим суть
 вакансии (длина заголовка ограничена, поэтому длинное не вместится). 5. Задать бюджет для своей кампании. Выбрать, сколько вы готовы заплатить за клик или показ
 объявления, а также установить общий бюджет для всей кампании. Общий бюджет должен быть не более 9000 рублей, а дневной бюджет рекомендуется указывать в размере 6000
 рублей. 6. Нажать "Запустить". 7. Отслеживать результаты. Вы можете отслеживать количество кликов и просмотров объявлений, а также другие показатели, чт

In [ ]:
gpt.answer_index(
    hr_promt,
    'Для чего нужны шаблоны вакансий?',
    hr_index
)

ОТВЕТ : 
  Шаблоны вакансий используются для быстрого и удобного создания новых вакансий в компании. Они помогают сократить время на заполнение необходимых полей при создании новых
 вакансий и отслеживании тех вакансий, которые были созданы на основе одного и того же шаблона. Кроме того, использование шаблонов вакансий обеспечивает единый стиль и
 уровень качества в описании вакансий, что упрощает процесс отбора подходящих кандидатов.


<h3><b>Методичка: карьерный трек AI разработчика</b></h3>

In [ ]:
#Методичка
AI_track_index = gpt.load_search_indexes('https://docs.google.com/document/d/1brdjCZN4zoy_drQUISUAX0541HfGMwcx3jlyhsGoWF4/edit#heading=h.k1ya7on3o1s9')


 ===========================================: 
Количество токенов в документе : 17134
ЦЕНА запроса: 0.0068536000000000005  $


In [ ]:
AI_track_promt = '''Ты методолог и преподаватель в компании, которая обучает AI разработчиков.
Перед тобой документ, в котором описаны варианты карьерного трека AI разработчика и методология подбора темы применения AI для компаний.
Твоя задача придумывать экзаменационные вопросы.
Тебе будут давать тему и твоя задача придумать несколько экзаменационных вопросов для студентов по этой теме исходя из документа.
Составляй вопросы максимально точно по документу, не придумывай ничего от себя.
Документ: '''

In [ ]:
gpt.answer_index(
    AI_track_promt,
    'Придумай 5 экзаменационных вопросов про подбор темы применения AI в компании',
    AI_track_index
)

ОТВЕТ : 
  1. Какой оптимальный алгоритм выбора темы AI-проекта в компании, описанный в документе №1, и как он был разработан? 2. Какие 4 вопроса предлагает использовать
 оптимальный алгоритм выбора темы AI-проекта, описанный в документе №1, и как они помогают найти идеи для проектов? 3. Какие задания и анализ бизнес-процессов ждут
 студентов в блоке #4 документа №2, связанном с выбором темы проекта? 4. Как выбрать тему применения AI в компании, согласно подходу, описанному в документе №4, и какие
 шаги следует предпринять для этого? 5. Каким образом можно выявить проблемы в бизнес-процессах компании для использования AI, согласно документу №3, и почему это важно
 для выбора темы проекта?


In [ ]:
gpt.answer_index(
    AI_track_promt,
    'Придумай 5 экзаменационных вопросов про карьерный трек AI разработчика',
    AI_track_index
)

ОТВЕТ : 
  1. Что включает в себя карьерный трек AI разработчика? 2. Почему важно начинать планирование карьерного трека с определения точки А? 3. Какой фактор надо учитывать при
 формировании карьерного трека? 4. Как влияет опыт программирования на карьерный трек AI разработчика? 5. Какое преимущество может дать правильно оформленный карьерный
 трек AI разработчику в его профессиональном развитии?


In [ ]:
gpt.answer_index(
    AI_track_promt,
    'Придумай 5 экзаменационных вопросов про виды AI задач',
    AI_track_index
)

ОТВЕТ : 
  1. Что такое карьерный трек в AI-разработке, и зачем он нужен? 2. Как определить свою Точку А при выборе карьерного пути в AI-разработке? 3. Какие факторы необходимо
 учитывать при определении текущего уровня дохода при выборе карьерного пути в AI-разработке? 4. Каковы наиболее востребованные профессиональные навыки в AI-разработке, и
 как их можно прокачать? 5. Почему важно определить свою мотивацию к изучению AI перед выбором карьерного пути, и как это можно сделать?


<h3><b>Саммаризация zoom презентации менеджера</b></h3>

In [ ]:
#Методичка
zoom_index = gpt.load_search_indexes('https://docs.google.com/document/d/1MuQ02a3Kz6ysDN43SM5YrURS2Mg2trLNrM_TyIElZLQ/edit?usp=sharing')


 ===========================================: 
Количество токенов в документе : 18071
ЦЕНА запроса: 0.007228400000000001  $


In [ ]:
zoom_promt = '''Инструкция по оценке диалога менеджера по продажам с клиентом

Перед тобой диалог менеджера по продажам с клиентом
Тебе надо проверить несколько критериев и заполнить отчёт
Не пиши общее сообщение, только отчёт по форме, форма отчёта будет в конце

Что надо проверить
1. Говорил ли клиент о потребности в трудоустройстве
2. Говорил ли клиент о потребности создать AI проект
3. Говорил ли клиент возражение о стоимости обучения
4. Говорил ли клиент возражение о времени на обучение

Форма отчёта
1. Потребность о трудоустройстве - было или нет
2. Потребность о создании AI проекте - было или нет
3. Возражение о стоимости - было или нет
4. Возражение о времени обучения - было или нет

Заполни отчёт и пришли в качестве ответа, коротко и ёмко'''

In [ ]:
gpt.answer_index(
    zoom_promt,
    'Пришли отчет',
    zoom_index
)

ОТВЕТ : 
  Отчет: 1. Потребность в трудоустройстве - было 2. Потребность в создании AI проектах - было 3. Возражение о стоимости - не было 4. Возражение о времени обучения - не
 было


In [ ]:
zoom_promt_2 = '''Ты менеджер контроля качества, твоя задача анализировать диалоги менеджеров по продажам с клиентами и готовить отчеты.
Компания продает курсы по AI разработке.
Перед тобой текст диалога сделанный с помощью распознавания речи из записи zoom презентации.
Из-за машинного распознавания речи, в тексте могут быть ошибки распознавания, учитывая это.
Твоя задача делать отчеты по данному диалогу по запросам пользователя.
Составляй вопросы максимально точно по диалогу, не придумывай ничего от себя.
Текст диалога: '''

In [ ]:
gpt.answer_index(
    zoom_promt_2,
    'Напиши отчет, какие были потребности названы клиентом',
    zoom_index
)

ОТВЕТ : 
  Отчет о потребностях клиента: Проанализировав диалог между менеджером и клиентом, можно выделить следующие потребности клиента: 1. Создание персонального шедевра через
 использование некоторых навыков, таких как вязание крючком. Клиент хочет определить свой стиль и подобрать уникальное изделие, которое будет модным. Также клиент
 интересовался возможностью оптимизации процесса создания персонального шедевра. 2. Понимание формата обучения. Клиент хотел привлекательный, интересный и необычный
 формат обучения, отличающийся от обычных методов. 3. Трудоустройство. Клиент интересовался процессом трудоустройства после обучения и возможностью увеличить свой опыт до
 необходимого для компаний уровня. Клиенту было предложено двадцать тридцать вакансий на выбор, с возможностью сделать для них тестовые задания и проходить собеседование.
 4. Использование искусственного интеллекта. В диалоге затрагивалась тема использования искусственного интеллекта в различных проектах, таких как отсле

In [ ]:
gpt.answer_index(
    zoom_promt_2,
    'Напиши отчет, почему клиенту вообще интересно изучение AI',
    zoom_index
)

ОТВЕТ : 
  Клиент проявляет интерес к AI в связи с поиском современных технологий, которые могут быть применимы в ее работе, она хочет разобраться в принципах функционирования
 искусственного интеллекта для того, чтобы использовать их в повседневной практике. В ходе диалога тема искусственного интеллекта неоднократно поднималась и клиент
 проявляла интерес к обучению на данной тематике, а также узнавала о возможностях применения AI в конкретных сферах деятельности.


In [ ]:
gpt.answer_index(
    zoom_promt_2,
    'Напиши отчет, какие возражения назвал клиент',
    zoom_index
)

ОТВЕТ : 
  Клиент не высказывал конкретных возражений, но задавал вопросы о цене и тарифах, а также интересовался, какие программы обучения предлагаются и возможности
 трудоустройства после курсов. Также клиент выразил свои сомнения относительно своей способности обучаться технологиям и проектам в сфере AI разработки, поскольку ранее
 не имел опыта работы в этой области. Тем не менее, клиент выразил интерес и готовность начать обучение с базовых программ и определенных тарифов обучения.


In [ ]:
gpt.answer_index(
    zoom_promt_2,
    'Напиши отчет, насколько проактивно менеджер действовал в диалоге',
    zoom_index
)

ОТВЕТ : 
  Отчет по проактивности менеджера в диалоге В данном диалоге можно отметить средний уровень проактивности со стороны менеджера. Проактивным моментом было предложение
 менеджером клиенту обучение на вакансии, где специалисты без опыта могут устроиться на работу. Также, менеджер предоставил клиенту возможность проходить обучение налогов
 быстрее, что так же можно отнести к проактивным действиям. Однако, в целом, менеджер в разговоре не использовал достаточно открытых вопросов, не проводил анализ
 интересов клиента, поэтому не всегда предлагал те решения, которые могли бы максимально соответствовать потребностям клиента. Таким образом, можно отметить, что менеджер
 мог быть более проактивен и активнее в предложении опций клиентам.


In [ ]:
gpt.answer_index(
    zoom_promt_2,
    'Напиши отчет, в котором будут цитаты из текста, которые говорят о потребностях клиента',
    zoom_index
)

ОТВЕТ : 
  Отчет о потребностях клиента в курсах по AI разработке. Клиент заинтересован в форматах обучения: "Какие у нас есть? Формат обучения, да?" Клиент также хочет иметь
 возможность загружать фотографии и получать консультации: "не париться, не сидеть там в интернете... искусственный интеллект отслеживал их в поведении на дороге...
 Постере, а уже код мог предложить... и По поводу обучения могу разговаривать. Вдруг там что то будет вам не понятно или не ясно?" Клиент также хочет получить возможности
 по трудоустройству после обучения: "Там наши Чары полностью упаковывают ваш фильм. Кейс помогают сделать, так что как будто у вас опыта. Именно По годам больше, чем есть
 на самом деле... вы Из этих вакансий выбираете те, которые вам только интересны. Там два три. Делайте для них тестовые задания. Проходите, собеседование у них, и вам уже
 предлагают работу." Клиент также обеспокоен ценой курсов: "Ну, пока у меня нет такого. Как то? Недешевое удовольствие. Вот и тут надо взвешивать к

<h3><b>Ответы на вопросы студентов</b></h3>

In [ ]:
#Методичка
conv_index = gpt.load_search_indexes('https://docs.google.com/document/d/1OzZvgAKPPozpin_D4g7UMC74US5yZnWznZRcQYGs2yM/edit?usp=sharing')


 ===========================================: 
Количество токенов в документе : 59860
ЦЕНА запроса: 0.023944  $


In [ ]:
conv_promt = '''Ты преподаватель по AI и опытный AI разработчик.
Ты умеешь объяснить все простыми словами для новичков.
Тебе будут писать новички, которые изучают AI.
И у тебя есть текст документа - это занятие по сверточным нейронным сетям.
Документ взят из google colab, в нем есть и текст, и исходники кода на python.
Ответь на вопросы обучающегося.
Составляй ответы точно по документу, не придумывай ничего от себя.
Документ: '''

In [ ]:
gpt.answer_index(
    conv_promt,
    '''Для чего мы используем в этом коде запись {:<20}?
      print('Класс: {:<20} {:3.0f}% сеть отнесла к классу {:<20} - {}'.format(class_labels[cls],
                                                                               100. * cm[cls, cls_pred],
                                                                               class_labels[cls_pred],
                                                                               msg))''',
    conv_index
)

ОТВЕТ : 
  Здесь используется запись {:<20} для выравнивания по левому краю текста в ячейках таблицы. К символам меток классов и сообщений об уверенности предсказания добавляются
 дополнительные пробелы справа до ширины ячеек, в нашем случае - до 20 символов. Слева от знака <: ставится знак <, так как мы выравниваем по левому краю.


In [ ]:
gpt.answer_index(
    conv_promt,
    '''Что здесь значит CLASS_LIST?
      compile_train_eval_model(model_conv,
                     	train_generator,
                     	validation_generator,
                     	test_generator,
                     	class_labels=CLASS_LIST,
                     	title='Сверточный классификатор')''',
    conv_index
)

NameError: ignored

<h3><b>Типирование по соционике</b></h3>

In [ ]:
gpt.answer_index(
    marketing_tg_post_promt,
    'Определи соционический тип автора документа',
    marketing_tg_post_index
)

ОТВЕТ : 
  К сожалению, нельзя определить соционический тип автора документа на основании предоставленной информации. Для определения соционического типа необходимо провести более
 глубокий анализ личности и поведенческих особенностей конкретного человека.


In [ ]:
socionics_promt = '''Ты - опытный типировщик по соционике.
Ты можешь типировать людей по одному отрывку текста, опираясь на особенности написания и используемых слов.
Тебе пришлют фрагмент текста человека, ты должен определить его соционический тип.
Фрагмент текста:'''

In [ ]:
gpt.answer_index(
    socionics_promt,
    'Сделай предположение о соционическом типе автора документа, выведи 3 самых вероятных типа по убыванию вероятности',
    marketing_tg_post_index
)

ОТВЕТ : 
  На основе данного отрывка документа сложно однозначно типировать автора, но я могу предположить, что вероятнее всего это человек, принадлежащий к интуитивно-логическому
 или интуитивно-этическому типу. Тройка наиболее вероятных типов по убыванию вероятности: 1. Интуитивно-логический (INTP) 2. Интуитивно-этический (INFJ) 3.
 Логико-интуитивный экстраверт (ENTP)


In [ ]:
gpt.answer_index(
    socionics_promt,
    'Сделай предположение о соционическом типе автора документа, выведи 3 самых вероятных типа по убыванию вероятности',
    hr_index
)

ОТВЕТ : 
  Исходя из отрывка текста, можно сделать вывод, что автор документа аккуратен, ориентирован на практичность и результативность, любит работу с данными и деталями, строго
 следует инструкциям и системно находится решение проблем. В качестве руководителя явно бы выбрал надежного, вдумчивого и дисциплинированного человека. Самые вероятные
 типы для такого аналитического и организованного человека могут быть: 1. Логико-сенсорный экстраверт (ЛСЭ) 2. Логико-интуитивный интроверт (ЛИИ) 3. Сенсорно-логический
 экстраверт (СЛЭ) Однако, на это стоит смотреть более глубоко и проанализировать автора более полно, чем по одному отрывку.
